In [1]:
from azureml.core import Experiment, Environment
from azureml.core.workspace import Workspace
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
import sys

In [2]:
ws = Workspace.from_config()

dataset = Dataset.get_by_name(ws, name='NASA TurboFan Databricks')
df_pd = dataset.to_pandas_dataframe()
df_pd.to_csv('data/turbofan.csv', index=False)

In [3]:
cpu_cluster_name = "cpuclst"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

aml_run_config = RunConfiguration()
aml_run_config.target = cpu_cluster
curated_environment = Environment.get(workspace=ws,name="sklearn_20_2")# name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
aml_run_config.environment = curated_environment


############# Experiement remote-gbr-turbofan ######################
experiement_name = 'gbr-turbofan'


exp = Experiment(workspace=ws, name=experiement_name)
src = ScriptRunConfig(source_directory='./', script='trainpy.py',run_config=aml_run_config)

run = exp.submit(src,tags={"python version": sys.version[0:6]})

run.wait_for_completion(show_output = True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
RunId: gbr-turbofan_1632722778_eb4c44fe
Web View: https://ml.azure.com/runs/gbr-turbofan_1632722778_eb4c44fe?wsid=/subscriptions/0a6d383c-9ce6-490d-89b8-47e6231192c3/resourcegroups/ai-workshop/workspaces/aiw-ml&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_d521fdd42a97d37c44755071b25b6237561dab1f95e6e6b5a5b585a4dda02390_d.txt

2021-09-27T06:08:20Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/aiw-ml/azureml/gbr-turbofan_1632722778_eb4c44fe/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/aiw-ml/azureml/gbr-turbofan_1632722778_eb4c44fe/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=24782 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/aiw-ml/azureml/gbr-turbofan_1632

Submitting /mnt/batch/tasks/shared/LS_root/mounts/clusters/cpuinstance/code/Users/joscholt/TurboFan/Python directory for run. The size of the directory >= 25 MB, so it can take a few minutes.
